In [1]:
import pandas as pd
from datasets import load_dataset
from datasets import Dataset

In [2]:
corpus = load_dataset("allenai/scifact","corpus")
dataset = load_dataset("allenai/scifact","claims")

Found cached dataset scifact (/home/milos.kosprdic.ivi/.cache/huggingface/datasets/allenai___scifact/corpus/1.0.0/15660e43ecfb3f7420850027005a63611abb2d401e9746b4059c1260745d9831)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset scifact (/home/milos.kosprdic.ivi/.cache/huggingface/datasets/allenai___scifact/claims/1.0.0/15660e43ecfb3f7420850027005a63611abb2d401e9746b4059c1260745d9831)


  0%|          | 0/3 [00:00<?, ?it/s]

In [81]:
corpus_strDict = dict()

for row in corpus['train']:
    
    if row['title'].strip().endswith('.'):
        title = row['title'].strip()
    elif row['title'].strip().endswith('?'):
        title = row['title'].strip()
    elif row['title'].strip().endswith('!'):
        title = row['title'].strip()
    elif row['title'].strip().endswith(','):
        title = row['title'].strip()[:-1]+'.'    
    elif row['title'].strip().endswith('☆'):
        title = row['title'].strip()[:-1]+'.'
    elif row['title'].strip().endswith('*'):
        title = row['title'].strip()[:-1]+'.'
    elif row['title'].strip().endswith(':'):
        title = row['title'].strip()[:-1]+'.'    
    elif row['title'].strip().endswith('�'):
        title = row['title'].strip()[:-1].rstrip()+'.'
    elif row['title'].strip().endswith('"'):
        title = row['title'].strip()[1:-1]+'.'
    elif row['title'].strip().endswith(')'):
        title = row['title'].strip()+'.'    
    else:
        title = row['title'].strip()+'.'

    abstract = str()
    
    for sent in row['abstract']:
        abstract = abstract + sent.strip() + ' '
    
    if abstract.rstrip().endswith(']'):
        abstract = abstract.rstrip().rpartition('[')[0].rstrip()
    elif abstract.rstrip().endswith(')'):
        abstract = abstract.rstrip().rpartition('(')[0].rstrip()
    elif abstract.rstrip().endswith('"'):
        abstract = abstract.rstrip()[:-3]+'"'+'.'
    elif abstract.rstrip().endswith('”'):
        abstract = abstract.rstrip()[:-3]+'”'+'.'
    elif abstract.rstrip().endswith('■'):
        abstract = abstract.rstrip()[:-2]
    else:
        abstract = abstract.rstrip()
    
    corpus_strDict[row['doc_id']] = {'title':title,'abstract':abstract.rstrip()}

## Transformation of the TRAIN subset of SciFACT dataset

In [107]:
id2label = {0: "NO_EVIDENCE", 1: "SUPPORT", 2: "CONTRADICT"}
label2id = {"NO_EVIDENCE": 0, "SUPPORT": 1, "CONTRADICT": 2}

#new_dataset = pd.DataFrame(columns=['claim','abstract','label'])
dataset_train_df = pd.DataFrame(columns=['claim', 'text', 'label'])

cnt = 0

for item in dataset['train']:
    
    if item['evidence_doc_id'] == '':
        for i in range(len(item['cited_doc_ids'])):
            doc_id = item['cited_doc_ids'][i]
            if item['evidence_label'] == '':
                label = "NO_EVIDENCE"
            claim = item['claim']
            abstract = corpus_strDict[doc_id]['title'].rstrip()+' '+corpus_strDict[doc_id]['abstract']
            #new_dataset.loc[cnt] = pd.Series({'claim':claim, 'abstract':abstract, 'label':label2id[label]})
            dataset_train_df.loc[cnt] = pd.Series({'claim': claim, 'text': abstract, 'label':label2id[label]})
            #print(dataset_train_df)
            cnt += 1
    
    else:
        doc_id = int(item['evidence_doc_id'])
        if item['evidence_label'] == 'SUPPORT':
            label = 'SUPPORT'
        elif item['evidence_label'] == 'CONTRADICT':
            label = 'CONTRADICT'
    
        claim = item['claim']
        abstract = corpus_strDict[doc_id]['title'].rstrip()+' '+corpus_strDict[doc_id]['abstract']
        #new_dataset.loc[cnt] = pd.Series({'claim':claim, 'abstract':abstract, 'label':label2id[label]})
        dataset_train_df.loc[cnt] = pd.Series({'claim': claim, 'text': abstract, 'label':label2id[label]})

        cnt += 1
    
dataset_train_df = dataset_train_df.reset_index(drop=True)

In [104]:
dataset_train_df_deduplicated = dataset_train_df.drop_duplicates()
dataset_train_df_deduplicated

,claim,text,label
0,0-dimensional biomaterials lack inductive prop...,New opportunities: the use of nanotechnologies...,0
1,1 in 5 million in UK have abnormal PrP positiv...,Prevalent abnormal prion protein in human appe...,2
2,1-1% of colorectal cancer patients are diagnos...,Relation between Medicare screening reimbursem...,0
3,10% of sudden infant death syndrome (SIDS) dea...,Sudden infant death syndrome. Despite declines...,0
4,32% of liver transplantation programs required...,Liver transplantation and opioid dependence. C...,1
...,...,...,...
1280,siRNA knockdown of A20 slows tumor progression...,A20 is overexpressed in glioma cells and may s...,1
1283,taa1 tar1 tar2 triple mutants in Arabidopsis d...,TAA1-Mediated Auxin Biosynthesis Is Essential ...,0
1284,β-sheet opening occurs during pleurotolysin po...,Conformational Changes during Pore Formation b...,1
1285,β1/Ketel is able to bind microtubules.,A Microtubule Interactome: Complexes with Role...,0


In [109]:
dataset_train_final = Dataset.from_pandas(dataset_train_df_deduplicated)

In [110]:
dataset_train_final

Dataset({
    features: ['claim', 'text', 'label', '__index_level_0__'],
    num_rows: 891
})

## Transformation of the VALIDATION subset of SciFACT dataset

In [114]:
id2label = {0: "NO_EVIDENCE", 1: "SUPPORT", 2: "CONTRADICT"}
label2id = {"NO_EVIDENCE": 0, "SUPPORT": 1, "CONTRADICT": 2}

#new_dataset = pd.DataFrame(columns=['claim','abstract','label'])
dataset_valid_df = pd.DataFrame(columns=['claim', 'text', 'label'])

cnt = 0

for item in dataset['validation']:
    
    if item['evidence_doc_id'] == '':
        for i in range(len(item['cited_doc_ids'])):
            doc_id = item['cited_doc_ids'][i]
            if item['evidence_label'] == '':
                label = "NO_EVIDENCE"
            claim = item['claim']
            abstract = corpus_strDict[doc_id]['title'].rstrip()+' '+corpus_strDict[doc_id]['abstract']
            #new_dataset.loc[cnt] = pd.Series({'claim':claim, 'abstract':abstract, 'label':label2id[label]})
            dataset_valid_df.loc[cnt] = pd.Series({'claim': claim, 'text': abstract, 'label':label2id[label]})
            #print(dataset_train_df)
            cnt += 1
    
    else:
        doc_id = int(item['evidence_doc_id'])
        if item['evidence_label'] == 'SUPPORT':
            label = 'SUPPORT'
        elif item['evidence_label'] == 'CONTRADICT':
            label = 'CONTRADICT'
    
        claim = item['claim']
        abstract = corpus_strDict[doc_id]['title'].rstrip()+' '+corpus_strDict[doc_id]['abstract']
        #new_dataset.loc[cnt] = pd.Series({'claim':claim, 'abstract':abstract, 'label':label2id[label]})
        dataset_valid_df.loc[cnt] = pd.Series({'claim': claim, 'text': abstract, 'label':label2id[label]})

        cnt += 1
    
dataset_valid_df = dataset_valid_df.reset_index(drop=True)

In [116]:
dataset_valid_df_deduplicated = dataset_valid_df.drop_duplicates()
dataset_valid_df_deduplicated

,claim,text,label
0,0-dimensional biomaterials show inductive prop...,New opportunities: the use of nanotechnologies...,0
1,"1,000 genomes project enables mapping of genet...",Rare Variants Create Synthetic Genome-Wide Ass...,1
3,1/2000 in UK have abnormal PrP positivity.,Prevalent abnormal prion protein in human appe...,1
4,5% of perinatal mortality is due to low birth ...,Estimates of global prevalence of childhood un...,0
5,A deficiency of vitamin B12 increases blood le...,Folic acid improves endothelial function in co...,0
...,...,...,...
446,Women with a higher birth weight are more like...,Intrauterine environments and breast cancer ri...,1
447,aPKCz causes tumour enhancement by affecting g...,Control of Nutrient Stress-Induced Metabolic R...,2
449,cSMAC formation enhances weak ligand signalling.,The stimulatory potency of T cell antigens is ...,1
450,mTORC2 regulates intracellular cysteine levels...,mTORC2 Regulates Amino Acid Metabolism in Canc...,1


In [117]:
dataset_valid_final = Dataset.from_pandas(dataset_valid_df_deduplicated)

In [118]:
dataset_valid_final

Dataset({
    features: ['claim', 'text', 'label', '__index_level_0__'],
    num_rows: 322
})

## Concatenation of transformed and deduplicated TRAIN and VALIDATION subsets into one dataset and making of the new train:valid:test split

In [120]:
dataset_all = pd.concat([dataset_train_df_deduplicated, dataset_valid_df_deduplicated])
dataset_all

,claim,text,label
0,0-dimensional biomaterials lack inductive prop...,New opportunities: the use of nanotechnologies...,0
1,1 in 5 million in UK have abnormal PrP positiv...,Prevalent abnormal prion protein in human appe...,2
2,1-1% of colorectal cancer patients are diagnos...,Relation between Medicare screening reimbursem...,0
3,10% of sudden infant death syndrome (SIDS) dea...,Sudden infant death syndrome. Despite declines...,0
4,32% of liver transplantation programs required...,Liver transplantation and opioid dependence. C...,1
...,...,...,...
446,Women with a higher birth weight are more like...,Intrauterine environments and breast cancer ri...,1
447,aPKCz causes tumour enhancement by affecting g...,Control of Nutrient Stress-Induced Metabolic R...,2
449,cSMAC formation enhances weak ligand signalling.,The stimulatory potency of T cell antigens is ...,1
450,mTORC2 regulates intracellular cysteine levels...,mTORC2 Regulates Amino Acid Metabolism in Canc...,1


In [125]:
df_train = pd.DataFrame()
for i in dataset_all.label.unique():
    df_train = df_train.append(dataset_all[dataset_all.label == i].sample(frac = 0.8))

df_rest = pd.concat([dataset_all, df_train, df_train]).drop_duplicates(keep=False)

df_valid = pd.DataFrame()
for i in df_rest.label.unique():
    df_valid = df_valid.append(df_rest[df_rest.label == i].sample(frac = 0.5))

df_test = pd.concat([df_rest, df_valid, df_valid]).drop_duplicates(keep=False)

df_train.shape[0] + df_valid.shape[0] + df_test.shape[0] == dataset_all.shape[0]

242


True

In [126]:
print('TRAINsize: ', df_train.shape[0])
print('VALIDsize: ', df_valid.shape[0])
print('TESTsize: ', df_test.shape[0])

TRAINsize:  971
VALIDsize:  120
TESTsize:  122


### Checking of the label ratio in new train:valid:test subsets

In [127]:
print(df_test[df_test.label==0].shape[0]/df_test.shape[0], df_test[df_test.label==1].shape[0]/df_test.shape[0], df_test[df_test.label==2].shape[0]/df_test.shape[0])
print(df_valid[df_valid.label==0].shape[0]/df_valid.shape[0], df_valid[df_valid.label==1].shape[0]/df_valid.shape[0], df_valid[df_valid.label==2].shape[0]/df_valid.shape[0])
print(df_train[df_train.label==0].shape[0]/df_train.shape[0], df_train[df_train.label==1].shape[0]/df_train.shape[0], df_train[df_train.label==2].shape[0]/df_train.shape[0])

0.36065573770491804 0.4180327868852459 0.22131147540983606
0.36666666666666664 0.4166666666666667 0.21666666666666667
0.364572605561277 0.41709577754891863 0.21833161688980432


In [128]:
df_train = df_train.sample(frac = 1).reset_index(drop=True)
df_valid = df_valid.sample(frac = 1).reset_index(drop=True)
df_test = df_test.sample(frac = 1).reset_index(drop=True)

konacno_train = Dataset.from_pandas(df_train)
konacno_valid = Dataset.from_pandas(df_valid)
konacno_test = Dataset.from_pandas(df_test)

In [29]:
df_train.to_csv('./SciFact_train.csv')

In [30]:
df_valid.to_csv('./SciFact_valid.csv')

In [31]:
df_test.to_csv('./SciFact_test.csv')

In [32]:
df_train.to_pickle('./SciFact_train.pkl', protocol=4)

In [33]:
df_valid.to_pickle('./SciFact_valid.pkl', protocol=4)

In [34]:
df_test.to_pickle('./SciFact_test.pkl', protocol=4)